## **CMSC 320 - Project 4**

# **Part 1: Getting some Data**

**Chosen Data**

I chose to use data of Baltimore Police Department Arrests.
I took the data from https://data.baltimorecity.gov/datasets/baltimore::bpd-arrests/explore?location=38.958098%2C-76.515650%2C8.00&showTable=true. There are multiple columns in the data, such as age, gender, race, and reason for arrest.

I also create a new column to display whether the arrest took place at night (8pm-5am).


In [61]:
import folium
import requests
import pandas as pd

#Baltimore Police Department Arrests
bpd_table = pd.read_csv("https://raw.githubusercontent.com/fabrizzio-perez/Baltimore-Police-Department-Arrests-Analysis/main/BPD_Arrests.csv", sep=',')

#CLEANING DATA

#drop any rows that do not have a latitude or longitude
df = bpd_table[bpd_table['Latitude'].notna()]
df = df[df['Longitude'].notna()]

#to_datetime
df['ArrestDateTime'] = pd.to_datetime(df['ArrestDateTime'])

#sort by most recent
df.sort_values('ArrestDateTime', ascending=False)

#create an hour column
df['hour'] = pd.to_datetime(df['ArrestDateTime']).dt.hour

#create a column to check if the arrest occured at night (Past or at 6PM, or before 5AM)
df['NightArrest'] = df['hour'].apply(lambda x: 1 if x < 5 or x >= 18 else 0)

display(df)

,X,Y,RowID,ArrestNumber,Age,Gender,Race,ArrestDateTime,ArrestLocation,IncidentOffence,...,ChargeDescription,District,Post,Neighborhood,Latitude,Longitude,GeoLocation,Shape,hour,NightArrest
0,1.407598e+06,592982.454220,1,22000022.0,18.0,M,B,2021-12-31 23:14:00+00:00,2800 EDMONDSON AVE,Unknown Offense,...,HGV,Western,721,Penrose/Fayette Street Outreach,39.2944,-76.6634,"(39.2944,-76.6634)",NaN,23,1
1,1.425804e+06,616913.737231,2,22000010.0,26.0,M,B,2021-12-31 22:09:00+00:00,5700 NORTHWOOD DR,Unknown Offense,...,HGV,Northern,523,Chinquapin Park,39.3599,-76.5987,"(39.3599,-76.5987)",NaN,22,1
6,1.415807e+06,599570.294917,7,21168306.0,44.0,M,B,2021-12-31 16:16:00+00:00,2200 LINDEN AVE,Unknown Offense,...,MOTOR VEHICLE UNLAW( STOLEN AU,Central,133,Reservoir Hill,39.3124,-76.6343,"(39.3124,-76.6343)",NaN,16,0
9,1.436607e+06,593105.703029,10,21168297.0,36.0,M,U,2021-12-31 14:50:00+00:00,100 JANNEY ST,Unknown Offense,...,AGG ASSAULT,Southeast,232,Kresson,39.2944,-76.5609,"(39.2944,-76.5609)",NaN,14,0
10,1.428927e+06,601702.345035,11,22000016.0,25.0,M,B,2021-12-31 12:51:00+00:00,2700 ST. LO DR,Unknown Offense,...,HGV,Northeast,431,Clifton Park,39.3181,-76.5879,"(39.3181,-76.5879)",NaN,12,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359700,1.408063e+06,605040.430923,359701,10000039.0,17.0,M,B,2010-01-01 00:05:00+00:00,2900 NORFOLK AV,54ARMED PERSON,...,HGV,Northwest,612,Park Circle,39.3275,-76.6616,"(39.3275,-76.6616)",NaN,0,1
359701,1.408063e+06,605040.430923,359702,10000036.0,43.0,M,B,2010-01-01 00:05:00+00:00,2900 NORFOLK AV,54ARMED PERSON,...,HGV,Northwest,612,Park Circle,39.3275,-76.6616,"(39.3275,-76.6616)",NaN,0,1
359702,1.401848e+06,594236.718020,359703,10000013.0,28.0,M,B,2010-01-01 00:01:00+00:00,900 WILDWOOD PW,79OTHER,...,HANDGUN VIOLATION,Southwest,815,Edmondson Village,39.2979,-76.6837,"(39.2979,-76.6837)",NaN,0,1
359703,1.429021e+06,599663.030860,359704,10000018.0,25.0,M,B,2010-01-01 00:01:00+00:00,1900 COLLINGTON AV,87NARCOTICS,...,DISTRIBUTION HERION,Eastern,331,South Clifton Park,39.3125,-76.5876,"(39.3125,-76.5876)",NaN,0,1


# **Part 2: Making a Map**

In [58]:
map_osm = folium.Map(location=[39.29, -76.61], zoom_start=13)
map_osm

# **Part 3: Combining Parts 1 and 2**

In [60]:
df.columns

"""
for index, arrest in df[df["Gender"] == "F"].iterrows():
    if arrest["ChargeDescription"] == "THEFT":
        folium.Marker(location=[arrest["Latitude"], arrest["Longitude"]],
            icon=folium.Icon(color='red')).add_to(map_osm)
        
    if arrest["ChargeDescription"] == 'POSS OF MARIJUANA':
        folium.Marker(location=[arrest["Latitude"], arrest["Longitude"]],
            icon=folium.Icon(color='green')).add_to(map_osm)


    if arrest["NightArrest"] == 0:
      if arrest["ChargeDescription"] == "POSS OF MARIJUANA":
        if arrest['Race'] == 'B':
          folium.Marker(location=[arrest["Latitude"], arrest["Longitude"]],
            icon=folium.Icon(color='red')).add_to(map_osm)

    if arrest["NightArrest"] == 0:
      if arrest["ChargeDescription"] == "POSS OF MARIJUANA":
        if arrest['Race'] == 'W':
          folium.Marker(location=[arrest["Latitude"], arrest["Longitude"]],
            icon=folium.Icon(color='red')).add_to(map_osm)

        if arrest['Race'] == 'B':
          folium.Marker(location=[arrest["Latitude"], arrest["Longitude"]],
            icon=folium.Icon(color='orange')).add_to(map_osm)
            

map_osm

"""


for index, arrest in df[df["Gender"] == "F"].iterrows():

    #output of Females Arrested for Marijuana Possesion at Night (Red for Black Females, Blue for White Females)
    if arrest["NightArrest"] == 1:
      if arrest["ChargeDescription"] == "POSS OF MARIJUANA":
        if arrest['Race'] == 'W':
          folium.Marker(location=[arrest["Latitude"], arrest["Longitude"]],
            icon=folium.Icon(color='blue')).add_to(map_osm)
        if arrest['Race'] == 'B':
          folium.Marker(location=[arrest["Latitude"], arrest["Longitude"]],
            icon=folium.Icon(color='red')).add_to(map_osm)

    #output for Females Arrested for Marijuana Possesion in the Morning/Afternoon (Purple for Black Females, Green for White Females)
    if arrest["NightArrest"] == 0:
      if arrest["ChargeDescription"] == "POSS OF MARIJUANA":
        if arrest['Race'] == 'W':
          folium.Marker(location=[arrest["Latitude"], arrest["Longitude"]],
            icon=folium.Icon(color='green')).add_to(map_osm)

        if arrest['Race'] == 'B':
          folium.Marker(location=[arrest["Latitude"], arrest["Longitude"]],
            icon=folium.Icon(color='purple')).add_to(map_osm)

map_osm

## **Interpretation of Data**

**My map displays all the arrests for white and black females for marijuana possesion by the Baltimore Police Department**. 

Red = Black Female Arrest at Night

Purple = Black Female Arrests NOT at night

Blue = White Female Arrests at Night

Green = White Female Arrests NOT at night

-
-

Overall, majority of female arrests for Marijuana Possesion in Baltimore County are more **concentrated in the heart of Baltimore**.

**Black Females are arrested for Marijuana Possesion more at night** than in the daytime. Whereas, **White Females are arrested for Marijuana Possesion more in the daytime** than at night.

One can conclude that overall, Black Females are disproportinately arrested for marijuana possesion compared to white females. Being singled out and possibly even targeted at night by police officers.
